In [ ]:
# class GumbelSelectorWeighted(nn.Module):
#     def __init__(self, input_size=6, k=3):
#         super().__init__()
#         self.k = k
#         self.logits = nn.Parameter(torch.randn(input_size))  # per la selezione
#         self.output_weights = nn.Parameter(torch.rand(k))    # pesi appresi

    

#   Introduzione su (nn.Module)

osservando la prima riga del codice troviamo **class GumbelSelectorWeighted(nn.moduel)**

    nn.Module è una classe base fornita dalla libreria PyTorch
- Rappresenta un  modulo (o un blocco) di una rete neurale. Un modulo può esse pensato come un contenutore per un insieme di parametri e funzioni
- Che possono esser utilizzati per elaborare i dati. Quando si crea una nuova classe che eredita da nn.Module, come in questo caso, 
- si sta creando un nuovo tipo di modulo che può essere utilizzato nella rete neurale. La classe GumbelSelectorWeighted contiene le definizioni 
- dei parametri e delle funzioni necessarie per implementare un selettore di pesi utilizzando la distribuzione di Gumbel.
 

_

    def __init__(self, input_size=6, k=3):
- È il costruttore della classe: viene eseguito quando crei il modello.
- input_size=6: il numero totale di input (es. [a1, ..., a6])
- k=3: quanti input vuoi selezionare automaticamente (es. 3 tra 6)


_

    super().__init__()
    self.k = k

- Chiama il costruttore della classe madre (nn.Module)
- Necessario per inizializzare correttamente la rete in PyTorch.
- Salva k come attributo di istanza, così puoi accedervi ovunque nella classe.


_

    self.logits = nn.Parameter(torch.randn(input_size))

- Qui viene creata una variabile allenabile (nn.Parameter) di lunghezza 6:
- Rappresenta i punteggi (logits) per ciascun input.
- Serve per decidere quali input selezionare
-  Più alto il logit → più alta la probabilità che l’input venga scelto dal Gumbel-Softmax.


_

    self.output_weights = nn.Parameter(torch.rand(k))

- Questa è una seconda variabile allenabile:
- È un vettore di lunghezza k (es. 3) → uno per ogni input selezionato
- Rappresenta i pesi che verranno applicati agli input scelti.
-  Se scegli [a1, a4, a6], allora output_weights sarà qualcosa tipo [1.4, 2.1, 0.9] da moltiplicare per i rispettivi valori.


In [ ]:
#     def forward(self, x, temperature=0.5):
#         # 1. Gumbel softmax campionamento
#         probs = gumbel_softmax(self.logits.unsqueeze(0), temperature=temperature, hard=False)
#         _, topk_indices = torch.topk(probs, self.k, dim=1)

#         # 2. Estrai solo i k input selezionati
#         selected_inputs = x[:, topk_indices[0]]  # shape: (batch_size, k)

#         # 3. Applica i pesi solo ai selezionati
#         weighted_sum = (selected_inputs * self.output_weights).sum(dim=1, keepdim=True)

#         return weighted_sum, topk_indices, self.output_weights

_

    def forward(self, x, temperature=0.5)
- **Questo è il metodo che calcola l’output del modello.**
- x: input (es. [[a1, a2, ..., a6]])
- temperature: controlla quanto è "netta" la selezione dei 3 input (più bassa = più netta)


_

    probs = gumbel_softmax(self.logits.unsqueeze(0), temperature=temperature, hard=False)
- **self.logits sono i punteggi appresi per ciascun input.**
- gumbel_softmax(...) → crea una distribuzione morbida (quasi probabilità) su 6 input
-  Alla fine ottieni un vettore di 6 numeri (sommati = 1), es: 
- **[0.01, 0.45, 0.02, 0.10, 0.39, 0.03]**


_


    _, topk_indices = torch.topk(probs, self.k, dim=1)
- Usa torch.topk per prendere gli indici dei k input migliori, cioè i 3 più probabili.
- Se probs = [0.01, 0.45, 0.02, 0.10, 0.39, 0.03], restituisce:
- **topk_indices = [1, 4, 3] → significa: scegli a2, a5, a4**


_

    selected_inputs = x[:, topk_indices[0]]  # shape: (batch_size, k)
- seleziona i valori del tensore x : topk_indices = [1, 4, 3]    →    selected_inputs = [[8, 33, 23]]



_

    weighted_sum = (selected_inputs * self.output_weights).sum(dim=1, keepdim=True)
-  Moltiplica ogni input selezionato per il suo peso appreso (output_weights)
- Poi li somma → questo è il valore stimato di S




# ESEMPIO PRATICO
- **x = [[10, 8, 15, 23, 33, 21]]** →
    - **topk_indices = [1, 4, 3]** → 
        - **selected_inputs = [8, 33, 23]** → 
            - **output_weights = [1.2, 0.8, 0.5]** → 
                - **weighted_sum = 8 x 1.2 + 33 x 0.8 + 23 x 0.5**